In [ ]:
import torch
from data_reading import WindowDataset
import os
import cv2
import numpy as np
from scipy import signal
import pickle
import json

In [ ]:
image_width = 128
image_height = 96
labels_folder = '../data/labels/'
imgs_folder = '../data/images/'
video_folder = '../data/training_videos/'
spectr_folder = '../data/spectrograms/'

In [ ]:
all_folders_img = set([dir for dir in os.listdir(labels_folder) if 'NW' in dir or 'W' in dir])
all_folders_lab = set([dir for dir in os.listdir(imgs_folder) if 'NW' in dir or 'W' in dir])
all_folders_video = set(['Folder '+dir.split('.')[0] for dir in os.listdir(video_folder) if 'NW' in dir or 'W' in dir])
all_folders = list(all_folders_img.intersection(all_folders_lab).intersection(all_folders_video))

In [ ]:
print(all_folders)

In [ ]:
dataset = WindowDataset(video_folder, labels_folder, imgs_folder)

In [ ]:
dataset.save(spectr_folder)

In [ ]:
all_spectr = {}
for file_ in all_folders:
    if 'NW' in file_ or 'W' in file_:
        file = file_[7:]+'.spectr'
        name = file.split('.')[0]
        print(f'{spectr_folder}{file}')
        with open(f'{spectr_folder}{file}','rb') as f:
            spectr = pickle.load(f)
        all_spectr[name] = spectr.permute(0,3,1,2)

In [ ]:
all_images = {}
for file in all_folders:
    if 'Folder' in file:
        name = file[7:]
        print(f'{imgs_folder}{file}')
        imgs = []
        img_dir_list = os.listdir(f'{imgs_folder}{file}')
        img_dir_list.sort()
        for imgfile in img_dir_list:
            if '.jpg' in imgfile:
                img = torch.tensor(cv2.cvtColor(cv2.imread(f"{imgs_folder}{file}/{imgfile}", cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB))
                img = img.permute(2,0,1).double()
                imgs.append(img)
        imgs = torch.stack(imgs)
        all_images[name] = imgs

In [ ]:
all_labs = {}
for file in all_folders:
    if 'Folder' in file:
        name = file[7:]
        print(f'{labels_folder}{file}')
        all_targets = []
        for interfolder in os.listdir(f'{labels_folder}{file}'):
            if interfolder.endswith('YOLO'):
                lab_list_dir = os.listdir(f'{labels_folder}{file}/{interfolder}')
                lab_list_dir.sort()
                prevCount = 0
                for labfile in lab_list_dir:
                    if '.txt' in labfile:
                        count = int(labfile.split('.')[0])
                        while count > prevCount+1:
                            targets = {'boxes':torch.tensor([[]]), 'labels':torch.tensor([])}
                            all_targets.append(targets)
                            prevCount+=1
                        prevCount = count
                        boxes = []
                        labels = []
                        with open(f"{labels_folder}{file}/{interfolder}/{labfile}") as f:
                            for line in f:
                                line = line.split()
                                label, rest = int(line[0]), line[1:]
                                xcenter, ycenter, w, h = map(float, rest)
                                xcenter, ycenter = int(xcenter*image_width), int(ycenter*image_height)
                                xmin, ymin = xcenter-int(w/2*image_width), ycenter-int(h/2*image_height)
                                boxes.append(torch.tensor([int(xmin), int(ymin), int(w*image_width), int(h*image_height)]))
                                labels.append(label)
                        if len(labels) == 0:
                            targets = {'boxes':torch.tensor([[]]), 'labels':torch.tensor([])}
                        else:
                            targets = {'boxes':torch.stack(boxes).float(), 'labels':torch.tensor(labels).long()+1}
                        all_targets.append(targets)
        all_labs[name] = all_targets

In [ ]:
with open('../data/spectro.data', 'wb') as f:
    pickle.dump(all_spectr, f)
with open('../data/images.data', 'wb') as f:
    pickle.dump(all_images, f)
with open('../data/labs.data', 'wb') as f:
    pickle.dump(all_labs, f)

In [ ]:
print(all_spectr['10 W'].shape)

In [ ]:
print(all_images['10 W'].shape)

In [ ]:
print(len(all_labs['10 W']))
print(all_labs['10 W'][0])
# why is there a count diff?

In [ ]:
imgs_folder = 'data/images/'
output = {'categories':[{'id':1, 'name':'window'}], 'images':[], 'annotations':[]}
count = 1
img_count = 1
new_spectr = {}
for name in all_labs.keys():
    labs = all_labs[name]
    imgs = all_images[name]
    spectrs = all_spectr[name]
    for i,lab in enumerate(labs):
        if i == 0 or i == len(labs)-1:
            continue
        if not (lab['boxes'].shape[0] == 1 and lab['boxes'].shape[1] == 0):
            for box in lab['boxes']:
                output['annotations'].append({})
                if len(box) == 0:
                    print(lab['boxes'])
                    continue
                boxx = [box[0].item(), box[1].item(), box[2].item(), box[3].item()]
                output['annotations'][-1] = {'id':count, 'image_id':img_count, 'category_id':1, 'bbox':boxx, 'iscrowd':0, 'area':boxx[2]*boxx[3]}
                count += 1
        new_spectr[img_count] = all_spectr[name][i-1]
        output['images'].append({})
        output['images'][-1] = {'id':img_count,'file_name':f'Folder {name}/{i:04d}.jpg'}
        img_count += 1



In [ ]:
with open('../data/new_spectro.data', 'wb') as f:
    pickle.dump(new_spectr, f)

In [ ]:
with open('../data/anno', 'w') as f:
    json.dump(output, f)

In [ ]:
with open('../data/anno', 'r') as f:
    output2 = json.load(f)
for img_anno in output2['images']:
    name = img_anno['file_name']
    assert(os.path.exists(f'../{imgs_folder}{name}'))
print(output2['annotations'])